#MNIST classification with RNN
Treat each row of MNIST image as an input vector at each time point. So, 32 rows will be fed sequentially. After that the output will be probablity vector for classification. This would be an example of a "sequence-to-vector" model of RNN 

In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

n_epochs = 10
batch_size_train = 200
batch_size_test = 1000

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

from google.colab import drive
drive.mount('/content/drive')

# Checking GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
cuda:0


In [0]:
from torch.utils.data import random_split

MNIST_training = torchvision.datasets.MNIST('/content/drive/My Drive/HIP2019/MNIST_dataset/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

MNIST_test_set = torchvision.datasets.MNIST('/content/drive/My Drive/HIP2019/MNIST_dataset/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

# create a training and a validation set
MNIST_training_set, MNIST_validation_set = random_split(MNIST_training, [55000, 5000])

train_loader = torch.utils.data.DataLoader(MNIST_training_set,batch_size=batch_size_train, shuffle=True)

validation_loader = torch.utils.data.DataLoader(MNIST_validation_set,batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(MNIST_test_set,batch_size=batch_size_test, shuffle=True)

In [3]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_data.shape,example_targets.shape)

torch.Size([1000, 1, 28, 28]) torch.Size([1000])


In [0]:
# Source: https://www.dezyre.com/recipes/run-basic-rnn-model-using-pytorch
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=28,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layers
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [5]:
rnn = RNN().to(device)
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)   # optimize all rnn parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

for epoch in range(10):
    for step, (x, y) in enumerate(train_loader):        # gives batch data
        b_x = x.view(-1, 28, 28).to(device)             # reshape x to (batch, time_step, input_size)
        b_y = y.to(device)                              # batch y

        output = rnn(b_x)                               # rnn output
        loss = loss_func(output, b_y)                   # cross entropy loss
        optimizer.zero_grad()                           # clear gradients for this training step
        loss.backward()                                 # backpropagation, compute gradients
        optimizer.step()                                # apply gradients

        if step % 50 == 0:
          with torch.no_grad():
            accuracy=0.0
            for validation_x, validation_y in validation_loader:
              validation_x = validation_x.view(-1, 28, 28).to(device)    # reshape x to (batch, time_step, input_size)
              validation_y = validation_y.to(device)                     # batch y
              validation_output = rnn(validation_x) 
              pred = validation_output.data.max(1, keepdim=True)[1]
              accuracy += 100.0*pred.eq(validation_y.data.view_as(pred)).sum() / float(len(validation_loader.dataset))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.item(), '| validation accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.2986 | validation accuracy: 16.66
Epoch:  0 | train loss: 0.5520 | validation accuracy: 82.62
Epoch:  0 | train loss: 0.2802 | validation accuracy: 91.70
Epoch:  0 | train loss: 0.2289 | validation accuracy: 92.92
Epoch:  0 | train loss: 0.2298 | validation accuracy: 93.98
Epoch:  0 | train loss: 0.1324 | validation accuracy: 94.96
Epoch:  1 | train loss: 0.1832 | validation accuracy: 95.44
Epoch:  1 | train loss: 0.1185 | validation accuracy: 96.20
Epoch:  1 | train loss: 0.1361 | validation accuracy: 96.54
Epoch:  1 | train loss: 0.1390 | validation accuracy: 96.00
Epoch:  1 | train loss: 0.1208 | validation accuracy: 96.84
Epoch:  1 | train loss: 0.1232 | validation accuracy: 96.24
Epoch:  2 | train loss: 0.0598 | validation accuracy: 96.52
Epoch:  2 | train loss: 0.1074 | validation accuracy: 96.64
Epoch:  2 | train loss: 0.0661 | validation accuracy: 96.72
Epoch:  2 | train loss: 0.0444 | validation accuracy: 96.78
Epoch:  2 | train loss: 0.0641 | validat

In [6]:
with torch.no_grad():
  accuracy=0.0
  for test_x, test_y in test_loader:
    test_x = test_x.view(-1, 28, 28).to(device)    # reshape x to (batch, time_step, input_size)
    test_y = test_y.to(device)                     # batch y
    test_output = rnn(test_x) 
    pred = test_output.data.max(1, keepdim=True)[1]
    accuracy += 100.0*pred.eq(test_y.data.view_as(pred)).sum() / float(len(test_loader.dataset))
  print('test accuracy: %.2f' % accuracy)

test accuracy: 97.78
